In [1]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

In [2]:
DIR="..\\..\\chunk\\18\\"
checks = pd.read_csv(f"{DIR}data.csv")
checks.drop(['name'], axis=1, inplace=True)

with open(f"{DIR}Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

checks.head()

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_9868\191298464.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  checks = pd.read_csv(f"{DIR}data.csv")


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [9]:
list(product_dict.items())[:3]

[('168308', '(197312) Пакет-майка 25см х 45см 906'),
 ('134832', '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404'),
 ('101384', '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404')]

In [10]:
checks.query('contact_id==197312')

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
1101056,2018-11-30,197312,324.0,86130.0,569.0,128.0,4389.0,"2,00"
1203680,2018-11-30,197312,324.0,65376.0,569.0,128.0,3328.0,"2,00"
2121985,2018-11-23,197312,324.0,163608.0,628.0,144.0,-1.0,"1,00"
2123704,2018-11-23,197312,324.0,41369.0,735.0,170.0,-1.0,"1,00"
2126573,2018-11-23,197312,324.0,60270.0,403.0,92.0,-1.0,"1,00"
...,...,...,...,...,...,...,...,...
16001323,2018-05-02,197312,324.0,47278.0,628.0,144.0,1226.0,"1,00"
16002347,2018-05-02,197312,324.0,100622.0,718.0,166.0,-1.0,"1,00"
16002948,2018-05-02,197312,324.0,44939.0,644.0,148.0,-1.0,"1,00"
16003083,2018-05-02,197312,324.0,147131.0,481.0,113.0,3888.0,"1,00"


In [11]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [14]:
checks.dropna(inplace=True)

In [15]:
checks[checks['sale_date_date'].str.startswith("20")][:4]

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0


In [19]:
checks[checks['sale_date_date'].str.startswith("20")!=True][:4]

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col


In [20]:
checks.shape

(20000000, 9)

In [21]:

checks = checks[checks['sale_date_date'].str.startswith("20")] #других нет годов,однако
checks.shape


(20000000, 9)

In [22]:
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию. Забьем на разорванные сессии
train, test = train_test_split(checks, test_size=0.3)

In [25]:
checks[:8]

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
6717817,2018-01-01,1918867,1517.0,124688.0,575.0,129.0,-1.0,"1,00",2018-01-01_1918867_1517.0
6666893,2018-01-01,843451,837.0,78119.0,648.0,148.0,-1.0,"1,00",2018-01-01_843451_837.0
6666892,2018-01-01,698663,138.0,41126.0,403.0,92.0,-1.0,"1,00",2018-01-01_698663_138.0
6666891,2018-01-01,753645,553.0,146960.0,906.0,205.0,-1.0,"1,00",2018-01-01_753645_553.0
6666890,2018-01-01,764682,138.0,131016.0,575.0,129.0,-1.0,"1,00",2018-01-01_764682_138.0
6666889,2018-01-01,730404,862.0,109932.0,569.0,128.0,1520.0,"1,00",2018-01-01_730404_862.0
6666888,2018-01-01,572634,99.0,144262.0,397.0,90.0,-1.0,"1,00",2018-01-01_572634_99.0
6666887,2018-01-01,784830,860.0,79490.0,700.0,162.0,-1.0,"1,00",2018-01-01_784830_860.0


In [23]:
train[:4]

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
10019023,2018-04-09,1063173,835.0,110310.0,431.0,101.0,-1.0,"1,00",2018-04-09_1063173_835.0
7065933,2018-03-01,497479,267.0,70966.0,738.0,170.0,-1.0,"1,00",2018-03-01_497479_267.0
15841310,2018-05-24,1041721,331.0,35007.0,736.0,170.0,-1.0,"1,00",2018-05-24_1041721_331.0
12830328,2018-04-10,1044448,313.0,19602.0,685.0,160.0,-1.0,"1,00",2018-04-10_1044448_313.0


In [24]:
test[:4]

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
13638121,2018-04-21,270556,264.0,108457.0,597.0,135.0,1348.0,"1,00",2018-04-21_270556_264.0
9652662,2018-03-27,177608,332.0,102814.0,738.0,170.0,-1.0,"1,00",2018-03-27_177608_332.0
16754402,2018-05-21,2493934,309.0,104644.0,595.0,135.0,-1.0,"1,00",2018-05-21_2493934_309.0
8370877,2018-03-14,63937,307.0,76973.0,-1.0,-1.0,-1.0,"1,00",2018-03-14_63937_307.0


In [7]:
# Обучим простые контетные рекомендации

# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences = [preprocess_txt(k) for k in product_dict.keys()]

# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
reverse_index_map = {}
counter = 0

for name in tqdm_notebook(product_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_names.build(10)

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1661400268.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(product_dict.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [8]:
reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [9]:
recommend(['52539', '110342'])

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\3531675664.py:12: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


['(35743) Шприцы одноразовые с иглой 5мл N1 762',
 '(56964) Линекс пор. лиоф. капс. №32 600',
 '(63827) Аркоксиа табл. п.п.о.90мг №7 647',
 '(87309) Минеральная вода Винцентка 0,7л 569',
 '(104883) Бальзам "Дикуля" Форте при заболеваниях позвоночника и суставов 75мл 718',
 '(67898) Фламакс форте табл. п.п.о 100мг N20 397',
 '(49051) Лавомакс таб.п.о.125мг №6 705',
 '(122344) Зизу Салфетки влажные для снятия макияжа с мицеллярной водой №10 496',
 '(180072) Норева Ксеродиан Плюс Крем для лица и тела пенящийся очищающий 500мл 496',
 '(56035) Фиточай Стевия 1г №20 611']

In [10]:
# Обучение эмбеддингов по последовательности
from gensim.models import Word2Vec

train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentences = []

In [11]:
# Сформируем последовательности

from tqdm import tqdm 
sentences = []
for group in tqdm_notebook(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1066363157.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(grouped.groups):


  0%|          | 0/5457574 [00:00<?, ?it/s]

In [13]:
modelW2V = Word2Vec(sentences, vector_size=10)

In [14]:
sentences[0]

['115617', '142296', '37070']

In [16]:
modelW2V.wv.similar_by_vector

<bound method KeyedVectors.similar_by_vector of <gensim.models.keyedvectors.KeyedVectors object at 0x000001E477776160>>

In [19]:
def recommend_w2v(items_list):
    current_vector = np.zeros(10)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in modelW2V.wv:
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
    return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]

In [20]:
recommend_w2v(['99821', '138583', '45321', '134475'])

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1051936176.py:10: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]


['(181271) Зубная паста Колгейт Древние секреты Безупречная Свежесть 75мл 529',
 '(67162) Габагамма капс. 300мг №50 641',
 '(187610) Гиалрипайер-10 Хондрорепарант гель гиалуроновый д/инъек.фл.0,8% 5мл №1 646',
 '(106823) ЛОкситан Шампунь Сила и Густота 300мл 476',
 '(192300) PL Перчатки латексные нестерильные р.M №10 675',
 '(67740) Аторвастатин таб.п.п.о.20мг №30 685',
 '(66406) Ополаскиватель д/полости рта Лакалют Уайт 300мл -1',
 '(35332) СТИКС Крем "Авокадо" 50 мл, уп №1 (150) 498',
 '(3434) Чай "Волшебная сила" [фильтр-пакеты 2г] N20 -1',
 '(183248) Катетер-баллон "Фолея" 2-х ходовой (№18) №1 761']